Домашнее задание № 5

Цель:

Разработать классификатор для определения спам/не 	спам сообщений с использованием различных методов предобработки текста и векторизации.

Задания: 

1. Подготовка данных:
- Загрузите датасет SMS сообщений, размеченных как спам или не спам.

https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

2. Предобработка текста:
- Реализуйте лемматизацию или стемминг для текстов сообщений.

3. Векторизация текста:
- Примените мешок слов (Bag of Words) и TF-IDF для векторизации текста.

4. Моделирование:
- Постройте модели для классификации сообщений как спам или не спам (модели на ваш выбор).
- Сравните результаты моделей с использованием различных методов векторизации и предобработки текста.

5. Сравнительный анализ:
- Оцените качество моделей с различными комбинациями предобработки и векторизации (сравните метрики ROC-AUC, F1-score, accuracy и т.д.).


In [6]:
# Загружаем всё необходимое для работы
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from matplotlib.colors import ListedColormap
from sklearn import metrics

# Строим загрузчик датасета, конвееры-предобработчики текста, преобразователи в матрицу, модели и сборщик метрик

def load_set_and_preclean(df):
    '''Задачи функции:
    1) Загрузка датасета;
    2) Сохранение признаков для дальнейшей работы;
    3) Кодирование данных в целевом признаке.
    '''
    # Загружаем SMS Spam Collection Dataset
    df = pd.read_csv(
        'https://raw.githubusercontent.com/akazachkov/ML_lesson_material/main/material_for_work/spam.csv',
        encoding = 'latin-1'
        )  # latin-1 используется, потому что UTF-8 (по умолчанию) вызовет ошибку

    df = df[['v1','v2']]  # Сохраняем только первые два признака
    df.columns = ['Target', 'Text']  # переименовываем их для удобства

    # Кодируем категориальные данные в числовые
    label_encoder = LabelEncoder()
    df["Target"] = label_encoder.fit_transform(df["Target"])
    return df

def preprocess_stemmer(text):
    '''
    Задачи, выполняемые обработчиком:
    1) Приведение текста к нижнему регистру;
    2) Исключение знаков препинания;
    3) Исключение стоп-слова;
    4) Стемминг.
    '''
    stemmer = PorterStemmer()
    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stopwords.words('english')]
    lemmatized_output = ' '.join([stemmer.stem(word) for word in filtered_tokens])
    return lemmatized_output

def preprocess_lemmatizer(text):
    '''
    Задачи, выполняемые обработчиком:
    1) Приведение текста к нижнему регистру;
    2) Исключение знаков препинания;
    3) Исключение стоп-слова;
    4) Лемматизация.
    '''
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stopwords.words('english')]
    lemmatized_output = ' '.join([lemmatizer.lemmatize(word, pos ='v') for word in filtered_tokens])
    return lemmatized_output

def train_test_split_df(df):
    '''Производим деление датафрейм на тренировочный и тестовый сеты, сбрасываем индексы'''
    train, test = train_test_split(df, test_size = 0.2, random_state = 42)
    train.reset_index(inplace = True)
    test.reset_index(inplace = True)
    return (train, test)

def TfidfVector(train, test):
    '''Преобразуем текстовые данные в матрицу TF-IDF с определёнными гиперпараметрами'''
    vectorizer = TfidfVectorizer(
        min_df = 8,  # Определяем минимальную частоту документов, в которых должен встречаться термин
        max_df = 0.4,  # Определяем максимальную долю документов, в которых может встречаться термин
        ngram_range = (1, 2)  # Указываем, чтобы в словаре появились биграммы
        )
    X_train = vectorizer.fit_transform(train.Text)
    X_test = vectorizer.transform(test.Text)
    y_train = train.Target.values
    y_test = test.Target.values
    return [X_train, X_test, y_train, y_test]

def CountVector(train, test):
    '''В этой функции вместо TF-IDF используем мешок слов (Bag of words) с такими же гиперпараметрами'''
    vectorizer = CountVectorizer(min_df = 8, max_df = 0.4, ngram_range = (1, 2))
    X_train = vectorizer.fit_transform(train.Text)
    X_test = vectorizer.transform(test.Text)
    y_train = train.Target.values
    y_test = test.Target.values
    return [X_train, X_test, y_train, y_test]

def models_building_and_evaluation(X_train, X_test, y_train, y_test):
    '''
    Задачи, выполняемые функцией:
    1) Создание моделей с определёнными параметрами;
    2) Обучение моделей;
    3) Оценка моделей и сохранение результатов в словарь;
    4) Добавление данных из словаря в DataFrame и вывод сводной информации.
    '''
    # Создаём листы, куда будем сохранять метрики
    precision = []
    recall = []
    f1_score = []
    roc_auc_score = []
    train_set_accuracy = []
    test_set_accuracy = []

    # Строим модели и задаём гиперпараметры
    classifiers = [MultinomialNB(alpha = 1.),
                   DecisionTreeClassifier(max_depth = 10),
                   RandomForestClassifier(n_estimators = 100, max_depth = 9, max_features = None),
                   KNeighborsClassifier(n_neighbors = 10, metric = 'cosine')]

    # Обучаем модели
    for cls in classifiers:
        cls.fit(X_train, y_train)

    # Оцениваем качество моделей
    for i in classifiers:
        pred_train = i.predict(X_train)
        pred_test = i.predict(X_test)
        precis = metrics.precision_score(y_test, pred_test)
        recal = metrics.recall_score(y_test, pred_test)
        f1_sc = metrics.f1_score(y_test, pred_test)
        roc_auc = metrics.roc_auc_score(y_test, pred_test)
        train_accuracy = metrics.accuracy_score(y_train, pred_train)
        test_accuracy = metrics.accuracy_score(y_test, pred_test)

        # Сохраняем метрики
        precision.append(precis)
        recall.append(recal)
        f1_score.append(f1_sc)
        roc_auc_score.append(roc_auc)
        train_set_accuracy.append(train_accuracy)
        test_set_accuracy.append(test_accuracy)

    # Собираем все метрики в один словарь
    data_results = {'Precision': precision,
                    'Recall': recall,
                    'F1_score': f1_score,
                    'ROC AUC': roc_auc_score,
                    'Accuracy on test set': test_set_accuracy,
                    'Accuracy on train set': train_set_accuracy}

    # Добавляем данные из словаря data_results в pandas DataFrame и выводим сводную информацию
    data_results_pd = pd.DataFrame(data_results, index = ["NaiveBayes",
                                                          "DecisionTreeClassifier",
                                                          "RandomForestClassifier",
                                                          "KNeighbours"])
    return data_results_pd.style.background_gradient(cmap = ListedColormap(["#fffaf0", "#fbceb1"]))

In [17]:
'''
Проходим весь процесс от загрузки датасета до вывода результатов.
Смотрим на результаты работы сочетания лемматизации и "мешка слов"
'''

# Создаём пустую переменную, куда будет загружен датасет
new_df = []

# Загружаем датасет и предварительно его обрабатываем
sms_df = load_set_and_preclean(new_df)

# Используем предобработчик текста, дополненный лемматизацией
sms_df['Text'] = sms_df['Text'].apply(preprocess_lemmatizer)

# Делим датафрейм на тренировочный и тестовый сеты
train, test = train_test_split_df(sms_df)

# Преобразуем текстовые данные с помощью техники "мешок слов"
X_train, X_test, y_train, y_test = CountVector(train, test)

# Создаём модели, обучаем и выводим результаты
data_results_lem_countvec = models_building_and_evaluation(X_train, X_test, y_train, y_test)
data_results_lem_countvec

,Precision,Recall,F1_score,ROC AUC,Accuracy on test set,Accuracy on train set
NaiveBayes,0.909091,0.866667,0.887372,0.926598,0.970404,0.982051
DecisionTreeClassifier,0.949495,0.626667,0.755020,0.810743,0.945291,0.963428
RandomForestClassifier,0.980000,0.653333,0.784000,0.825630,0.951570,0.969037
KNeighbours,0.936364,0.686667,0.792308,0.839706,0.951570,0.961409


In [21]:
'''
Проходим весь процесс от загрузки датасета до вывода результатов.
Смотрим на результаты работы сочетания стемминга и TF-IDF
'''

# Создаём пустую переменную, куда будет загружен датасет
new_df = []

# Загружаем датасет и предварительно его обрабатываем
sms_df = load_set_and_preclean(new_df)

# Используем предобработчик текста, дополненный стеммингом
sms_df['Text'] = sms_df['Text'].apply(preprocess_stemmer)

# Делим датафрейм на тренировочный и тестовый сеты
train, test = train_test_split_df(sms_df)

# Преобразуем текстовые данные в матрицу TF-IDF
X_train, X_test, y_train, y_test = TfidfVector(train, test)

# Создаём модели, обучаем и выводим результаты
data_results_stem_tf = models_building_and_evaluation(X_train, X_test, y_train, y_test)
data_results_stem_tf

,Precision,Recall,F1_score,ROC AUC,Accuracy on test set,Accuracy on train set
NaiveBayes,0.967480,0.793333,0.871795,0.894594,0.968610,0.980256
DecisionTreeClassifier,0.900826,0.726667,0.804428,0.857116,0.952466,0.975768
RandomForestClassifier,0.903226,0.746667,0.817518,0.867116,0.955157,0.975544
KNeighbours,0.955752,0.720000,0.821293,0.857409,0.957848,0.965672


In [22]:
'''
Проходим весь процесс от загрузки датасета до вывода результатов.
Смотрим на результаты работы сочетания лемматизации и TF-IDF
'''

# Создаём пустую переменную, куда будет загружен датасет
new_df = []

# Загружаем датасет и предварительно его обрабатываем
sms_df = load_set_and_preclean(new_df)

# Используем предобработчик текста, дополненный лемматизацией
sms_df['Text'] = sms_df['Text'].apply(preprocess_lemmatizer)

# Делим датафрейм на тренировочный и тестовый сеты
train, test = train_test_split_df(sms_df)

# Преобразуем текстовые данные в матрицу TF-IDF
X_train, X_test, y_train, y_test = TfidfVector(train, test)

# Создаём модели, обучаем и выводим результаты
data_results_lem_tf = models_building_and_evaluation(X_train, X_test, y_train, y_test)
data_results_lem_tf

,Precision,Recall,F1_score,ROC AUC,Accuracy on test set,Accuracy on train set
NaiveBayes,0.967480,0.793333,0.871795,0.894594,0.968610,0.980031
DecisionTreeClassifier,0.885246,0.720000,0.794118,0.852746,0.949776,0.975544
RandomForestClassifier,0.910569,0.746667,0.820513,0.867634,0.956054,0.975544
KNeighbours,0.955752,0.720000,0.821293,0.857409,0.957848,0.964999
